**Programador:** Marco Gutierrez. Si se tiene alguna consulta, mandarla al siguiente [email](mailto:mgutierrezc@up.edu.pe)

# Obteniendo la información para el Scrapping

Importamos las librerias para el scrapping

In [2]:
import pandas as pd
import string
import googlemaps
import numpy as np
import os
import re

## Importando las provincias/distritos a analizar

Primero, leeremos un archivo que contiene los elementos de los ubigeos del INEI. Se puede descargar entrando [aquí](http://webinei.inei.gob.pe:8080/sisconcode/proyecto/index.htm?proyectoTitulo=UBIGEO&proyectoId=3) y dando click a la opción **Excel** en la pestaña *Busqueda por Ubicación Geográfica*

In [3]:
ubigeos = pd.read_excel('ubigeo_inei.xls',skiprows=1)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,01 Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,02 Asunción
4,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,03 Balsas


Necesitamos las provincias y distritos de esta base, así que limpiaremos los valores de estas variables

In [4]:
ubigeos["DISTRITO"] = ubigeos["DISTRITO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["PROVINCIA"] = ubigeos["PROVINCIA"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["DEPARTAMENTO"] = ubigeos["DEPARTAMENTO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,Amazonas,NaN,NaN,,
1,NaN,Amazonas,NaN,NaN,Chachapoyas,
2,NaN,Amazonas,NaN,NaN,Chachapoyas,Chachapoyas
3,NaN,Amazonas,NaN,NaN,Chachapoyas,Asunción
4,NaN,Amazonas,NaN,NaN,Chachapoyas,Balsas


Necesitaremos estas dos columnas como listas, pues después las usaremos para crear un DataFrame donde guardaremos los resultados del scrapping

In [5]:
# Creando las listas
distritos = list(ubigeos['DISTRITO'] + ", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])
for_removal_dist = list(", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])

Limpiando las listas

In [6]:
# Eliminando duplicados
distritos = set(distritos)
for_removal_dist = set(for_removal_dist)

# Eliminando algunos elementos inutiles ('', nan)
distritos = list(filter(lambda x: str(x) != 'nan', distritos))

distritos = [distr for distr in distritos if distr not in for_removal_dist]
distritos.remove('DISTRITO, PROVINCIA, DEPARTAMENTO')

Eliminando las tildes

In [7]:
# Definimos una función para eliminar las tildes
def normalize(s):
    replacements = (
        ("á", "a"),
        ("à", "a"),
        ("é", "e"),
        ("è", "e"),
        ("í", "i"),
        ("ì", "i"),
        ("ó", "o"),
        ("ò", "o"),
        ("ú", "u"),
        ("ù", "u")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [8]:
# La aplicamos a cada elemento de nuestras distritos
nro_distrs = len(distritos)
for i in range(nro_distrs):
    distritos[i] = normalize(distritos[i])
    distritos[i] = distritos[i].split(",")
    
    # Eliminando los espacios dentro de los elementos de un distrito único (distr, prov, dept)
    item_distr = len(distritos[i])
    empty_distr = []
    for j in range(item_distr):
        distritos[i][j] = distritos[i][j].strip()
        empty_distr.append(distritos[i][j])
        
    # Joining our separated districts
    distritos[i] = ','.join(empty_distr)

# Scrapping de Distancias y Tiempos de viaje

Para realizar este scrapping, se tomará como referencia esta [guía](https://www.datahubbs.com/google-maps-python/)

## Matrices base a nivel de distritos

Crearemos nuestra matriz de la siguiente manera

In [9]:
first_col = ["Origin Province\Destination Province"]
dist_distances = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_times = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_distances

,Origin Province\Destination Province,"Ranracancha,Chincheros,Apurimac","Anco,La Mar,Ayacucho","Roble,Tayacaja,Huancavelica","Huancavelica,Huancavelica,Huancavelica","Andahuaylillas,Quispicanchi,Cusco","Vichayal,Paita,Piura","Ajoyani,Carabaya,Puno","Posic,Rioja,San Martin","Manzanares,Concepcion,Junin",...,"Ucuncha,Bolivar,La Libertad","Chanchamayo,Chanchamayo,Junin","Surquillo,Lima,Lima","Pachangara,Oyon,Lima","Ocoña,Camana,Arequipa","Victor Larco Herrera,Trujillo,La Libertad","Alonso de Alvarado,Lamas,San Martin","Candarave,Candarave,Tacna","Yarumayo,Huanuco,Huanuco","Pancan,Jauja,Junin"
0,"Ranracancha,Chincheros,Apurimac",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Anco,La Mar,Ayacucho",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Roble,Tayacaja,Huancavelica",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Huancavelica,Huancavelica,Huancavelica",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Andahuaylillas,Quispicanchi,Cusco",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,"Victor Larco Herrera,Trujillo,La Libertad",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1870,"Alonso de Alvarado,Lamas,San Martin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871,"Candarave,Candarave,Tacna",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1872,"Yarumayo,Huanuco,Huanuco",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [10]:
# Como queremos iterar sobre los nombres de las provincias
origen_destino_dist=[]

for dist_row in distritos:
    origen_destino_dist.append(dist_row+","+"Peru")

origen_destino_dist

['Ranracancha,Chincheros,Apurimac,Peru',
 'Anco,La Mar,Ayacucho,Peru',
 'Roble,Tayacaja,Huancavelica,Peru',
 'Huancavelica,Huancavelica,Huancavelica,Peru',
 'Andahuaylillas,Quispicanchi,Cusco,Peru',
 'Vichayal,Paita,Piura,Peru',
 'Ajoyani,Carabaya,Puno,Peru',
 'Posic,Rioja,San Martin,Peru',
 'Manzanares,Concepcion,Junin,Peru',
 'Luricocha,Huanta,Ayacucho,Peru',
 'Lonya Grande,Utcubamba,Amazonas,Peru',
 'Lircay,Angaraes,Huancavelica,Peru',
 'Acopampa,Carhuaz,Ancash,Peru',
 'Tate,Ica,Ica,Peru',
 'Luis Carranza,La Mar,Ayacucho,Peru',
 'Margos,Huanuco,Huanuco,Peru',
 'Gorgor,Cajatambo,Lima,Peru',
 'Jesus Maria,Lima,Lima,Peru',
 'Simbal,Trujillo,La Libertad,Peru',
 'Jayanca,Lambayeque,Lambayeque,Peru',
 'Tingo,Luya,Amazonas,Peru',
 'Sayla,La Union,Arequipa,Peru',
 'Yungar,Carhuaz,Ancash,Peru',
 'San Jose del Alto,Jaen,Cajamarca,Peru',
 'Tambo,La Mar,Ayacucho,Peru',
 'Santa Rosa de Alto Yanajanca,Marañon,Huanuco,Peru',
 'La Esperanza,Trujillo,La Libertad,Peru',
 'Chuschi,Cangallo,Ayacucho,Pe

## ¿Cómo usar el paquete `googlemaps`?

Ahora que ya tenemos todos los datos listos, nos falta activar la clase `googlemaps` para que podamos extraer tiempos y distancias. Ello se puede hacer así:

1. Importamos la librería googlemaps (Hecho al inicio de este jupyter)
1. Activamos el API `Distance Matrix API` en Google Cloud Platform (se puede encontrar una guía [aquí](https://www.youtube.com/watch?v=n1UorU1PALk))
1. Creamos una API key para poder usar el item anterior (guía [aquí](https://www.youtube.com/watch?v=2_HZObVbe-g))
1. Activaremos el cobro por el uso del API (Nos daran una prueba gratuita. La guía está [aquí](https://support.google.com/googleapi/answer/6158867?hl=en))

Sin un API key no podrán seguir el proceso, pero es de uso **personal**. Por ello, nunca la debemos escribir directamente en nuestro código; sin embargo, aprenderemos como utilizarla manteniendola privada y lo haremos usando **variables del sistema**

En el Menú de Inicio de Windows, busquen "environment" o "variables". Escojan la opción **"Editar las variables de entorno del sistema"**
![](images\env1.jpg)

Abranlo y entren a la pestaña `Opciones Avanzadas`

![](images\env2.jpg)

Hagan click a la opción `Variables de entornno`
![](images\env3.jpg)

Vayan a la sección `Variables de entorno` y denle click a **Nueva...**
![](images\env4.jpg)

Coloquen el nombre que prefieran (en mi caso, usé  `maps_key`) y, como valor, la API Key correspondiente (no la deben colocar entre comillas)
![](images\env5.jpg)

Denle Aceptar a esta ventana y luego a las anteriores hasta que se cierren todas. Cuando estén cerradas, reinicien su computadora y ya deberían poder usarla

## Usando `googlemaps` para crear una función que calcule distancias/tiempos

Cuando ya tengan su API Key lista, llamenla de la siguiente manera

In [11]:
key = os.environ.get('maps_key')

Ahora, activaremos la clase google maps usando nuestro key

In [12]:
gmaps = googlemaps.Client(key)

In [13]:
def matrix_blocks(first_row, first_column, final_row_plus_one, final_col, df_distances, df_times, origin_dest):
    """
    Función para completar la matriz por bloques que comiencen desde "first_row" y "first_column" hasta "iterations_row" 
    y "iterations_column". Se creó porque ejecutar el sgte codigo para todas las celdas toma mucho tiempo en ejecutarse.
    
    Para hacer más eficiente la tarea, se puede ejecutar esto por partes hasta completarlo. Además, permite convertir la tarea
    en una grupal. Un equipo puede hacer que cada uno de sus miembros complete ciertos bloques (e.g. linea 1 hasta 12, otro 
    linea 12 hasta 24, y así sucesivamente)
    """
    # Codigo base para cada fila (comenzando con la 0)
    for row in range(first_row, final_row_plus_one):
        print(row)
        for col in range(first_column, final_col):
            # Definiendo un índice para nuestra lista con los origenes/destinos en función a la columna del dataframe
            # en la que queremos el resultado
            index_destination = col-1

            # Si las filas y columnas son consecutivas, las distancias y tiempo son de un sitio al mismo sitio, asi que 
            # debemos controlar ese factir
            if col-1==row:
                df_distances.iloc[row,col] = '0 km'
                df_times.iloc[row,col] = '0 mins'

            else:
                # Obteniendo las distancias/tiempos correspondientes: 
                dist_time_mat = gmaps.distance_matrix(origins=origin_dest[row], destinations=origin_dest[index_destination], mode='driving')
                distance_duration = dist_time_mat['rows'][0]['elements'][0]

                # No siempre es posible viajar de un lugar a otro por restricciones geográficas, así que debemos definir qué ocurre
                # en casos así
                if distance_duration.get('distance'):
                    df_distances.iloc[row,col] = distance_duration['distance']['text']
                else:
                    df_distances.iloc[row,col] = '-'

                if distance_duration.get('duration'):
                    df_times.iloc[row,col] = distance_duration['duration']['text']
                else:
                    df_times.iloc[row,col] = '-'            

                # Para debuggear:
                # print('Origen: '+origen_destino[row]+' | Destino: '+origen_destino[index_destination]) 
                # print(prov_distances.iloc[row,col])

### Aplicando la función a las Matrices de Distritos

In [20]:
# La primera fila puede tomar valores comenzando desde 0 
primera_fila = 41
# Si quieres una determinada fila final, por ej, fila 2 (comenzando en 0), súmale uno a su valor 
# y registralo aquí (ult_fila_mas_uno = 3)
ult_fila_mas_uno = 51
primera_col = 1
ult_col = len(distritos)+1

matrix_blocks(primera_fila, primera_col, ult_fila_mas_uno, ult_col, dist_distances, dist_times, origen_destino_dist)

41
42
43
44
45
46
47
48
49
50


In [21]:
dist_distances.head()

,Origin Province\Destination Province,"Ranracancha,Chincheros,Apurimac","Anco,La Mar,Ayacucho","Roble,Tayacaja,Huancavelica","Huancavelica,Huancavelica,Huancavelica","Andahuaylillas,Quispicanchi,Cusco","Vichayal,Paita,Piura","Ajoyani,Carabaya,Puno","Posic,Rioja,San Martin","Manzanares,Concepcion,Junin",...,"Ucuncha,Bolivar,La Libertad","Chanchamayo,Chanchamayo,Junin","Surquillo,Lima,Lima","Pachangara,Oyon,Lima","Ocoña,Camana,Arequipa","Victor Larco Herrera,Trujillo,La Libertad","Alonso de Alvarado,Lamas,San Martin","Candarave,Candarave,Tacna","Yarumayo,Huanuco,Huanuco","Pancan,Jauja,Junin"
0,"Ranracancha,Chincheros,Apurimac",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Anco,La Mar,Ayacucho",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Roble,Tayacaja,Huancavelica",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Huancavelica,Huancavelica,Huancavelica",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Andahuaylillas,Quispicanchi,Cusco",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
ultima_fila = ult_fila_mas_uno - 1
dist_distances.to_csv('district_data\\dist_distances_ini-{}_fin-{}.csv'.format(primera_fila, ultima_fila), index=False)
dist_times.to_csv('district_data\\dist_times_ini-{}_fin-{}.csv'.format(primera_fila, ultima_fila), index=False)